In [1]:
import requests
import pandas as pd

In [14]:
BOT_FX = "https://gateway.api.bot.or.th/Stat-ExchangeRate/v2/MONTHLY_AVG_EXG_RATE/"
BOT_INT_AVG = "https://gateway.api.bot.or.th/LoanRate/v2/avg_loan_rate/"
BOT_INT_LOAN = "https://gateway.api.bot.or.th/LoanRate/v2/loan_rate/"
CPI = "https://dataapi.moc.go.th/cpiu-indexes"

FX_API_KEY = "eyJvcmciOiI2NzM1NzgwZWM4YzFlYjAwMDEyYTM3NzEiLCJpZCI6ImZlZGY5YzI1ZWZlODRjMzNiMzQzNmUwNzQ4OTAxMTIyIiwiaCI6Im11cm11cjEyOCJ9"
INT_API_KEY = "eyJvcmciOiI2NzM1NzgwZWM4YzFlYjAwMDEyYTM3NzEiLCJpZCI6ImRiYzQyMzA5ZmRiZTQ3YzZiZWZjZDdkYTFmMGU4ZjNiIiwiaCI6Im11cm11cjEyOCJ9"


### FX

In [18]:
headers_fx = {"Authorization": FX_API_KEY}
params_fx = {"start_period": "2023-01", "end_period": "2024-11"}
# ส่ง request ไป get ข้อมูลจาก BOT_FX
res_fx = requests.get(BOT_FX, headers=headers_fx, params=params_fx)
fx = res_fx.json()
# สกัดข้อมูลจาก data_detail
fx_detail = fx['result']['data']['data_detail']
fx_df = pd.DataFrame(fx_detail)
# ลบ column ที่ไม่ใช้
fx_data = fx_df.drop(columns=['currency_name_eng'])

fx_data
fx_data.to_csv("./fx_data.csv", index=False)

In [15]:
headers_fx = {"Authorization": FX_API_KEY}
params_fx = {"start_period": "2023-01", "end_period": "2024-11"}
# ส่ง request ไป get ข้อมูลจาก BOT_FX
res_fx = requests.get(BOT_FX, headers=headers_fx, params=params_fx)
fx = res_fx.json()
fx

{'result': {'timestamp': '2025-11-10 16:13:27',
  'api': 'Monthly Average Exchange Rate - THB / Foreign Currency',
  'data': {'data_header': {'report_name_eng': 'Rates of Exchange of Commercial Banks in Bangkok Metropolis (2002-present)',
    'report_name_th': 'อัตราแลกเปลี่ยนเฉลี่ยของธนาคารพาณิชย์ในกรุงเทพมหานคร (2545-ปัจจุบัน)',
    'report_uoq_name_eng': '(Unit: Baht / 1 Unit of Foreign Currency)',
    'report_uoq_name_th': '(หน่วย: บาท ต่อ 1 หน่วยเงินตราต่างประเทศ)',
    'report_source_of_data': [{'source_of_data_eng': 'Bank of Thailand',
      'source_of_data_th': 'ธนาคารแห่งประเทศไทย'}],
    'report_remark': [{'report_remark_eng': 'Since Nov 16, 2015 the data regarding Buying Transfer Rate of PKR has been changed to Buying Rate using Foreign Exchange Rates (LSEG) with Bangkok Market Crossing.',
      'report_remark_th': 'ตั้งแต่วันที่ 16 พ.ย. 2558 ข้อมูลในอัตราซื้อเงินโอนของสกุล PKR ได้เปลี่ยนเป็นอัตราซื้อที่ใช้อัตราในตลาดต่างประเทศ (LSEG) คำนวณผ่านอัตราซื้อขายเงินดอลลาร์ สรอ. ใน

# CPI

In [12]:
import requests
import urllib3

# ปิดคำเตือน SSL
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_cpiu_data(region_id, index_id, from_year, to_year):
    url = "https://dataapi.moc.go.th/cpiu-indexes"
    params = {
        'region_id': region_id,
        'index_id': index_id,
        'from_year': from_year,
        'to_year': to_year
    }
    
    try:
        response = requests.get(url, params=params, verify=False, timeout=30)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

# ใช้งาน
CPI = get_cpiu_data(5, '0000000000000000', 2023, 2024)

# Clean ข้อมูล CPI
CPI_df = pd.DataFrame(CPI)
CPI_df = CPI_df.drop(columns=['index_id','index_description','region_id','region_name'])
CPI_df['year_month'] = CPI_df['year'].astype(str) + '-' + CPI_df['month'].astype(str).str.zfill(2)
CPI_clean = CPI_df.drop(columns=['year','month'])

CPI_clean

Error: HTTPSConnectionPool(host='dataapi.moc.go.th', port=443): Read timed out. (read timeout=30)


KeyError: "['index_id', 'index_description', 'region_id', 'region_name'] not found in axis"

## Interest

In [39]:
    def fetch_data(api_url, api_key, start_date, end_date):
        """เรียก API ดึงข้อมูล"""
        params = {
            'start_period': start_date,
            'end_period': end_date
        }

        headers = {
            'Authorization': api_key
        }

        try:
            response = requests.get(api_url, params=params, headers=headers, timeout=30)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {start_date} to {end_date}: {e}")
            return None

    def extract_details(response_data):
        """ดึง data_detail จาก response"""
        if response_data and 'result' in response_data:
            data = response_data.get('result', {}).get('data', {})
            return data.get('data_detail', [])
        return []

    def fetch_monthly_data(api_url, api_key, start_date, end_date, delay=1):
        """ดึงข้อมูลทีละเดือนและรวมกัน"""
        date_ranges = get_date_ranges(start_date, end_date)
        all_details = []


        for i, (start, end) in enumerate(date_ranges, 1):
            print(f"  [{i}/{len(date_ranges)}] {start} ถึง {end}...", end=' ')
            
            response_data = fetch_data(api_url, api_key, start, end)
            details = extract_details(response_data)

            if details:
                all_details.extend(details)
                print(f"✓ {len(details)} รายการ")
            else:
                print("⚠️ ไม่พบข้อมูล")

            # หน่วงเวลาระหว่างการเรียก API
            if i < len(date_ranges):
                time.sleep(delay)

        return all_details

    # ตั้งค่า API
    API_URL = BOT_INT_AVG
    API_KEY = INT_API_KEY
    start_date = "2023-01-01"
    end_date = "2024-11-30"

    # ดึงข้อมูล
    data_details = fetch_monthly_data(API_URL, API_KEY, start_date, end_date, delay=1)

    # Clean ข้อมูล int
    interest_df = pd.DataFrame(data_details)
    numeric_cols = ['mor', 'mlr', 'mrr', 'ceiling_rate', 'default_rate', 'creditcard_min', 'creditcard_max']
    
    for col in numeric_cols:
        if col in interest_df.columns:
            interest_df[col] = pd.to_numeric(interest_df[col], errors='coerce')

    # เพิ่ม year และ month จาก period
    interest_df['period'] = pd.to_datetime(interest_df['period'])
    interest_df['year'] = interest_df['period'].dt.year
    interest_df['month'] = interest_df['period'].dt.month

    # Group and aggregate
    available_cols = [col for col in numeric_cols if col in interest_df.columns]
    monthly_avg_interest_df = interest_df.groupby(['year', 'month'])[available_cols].mean().reset_index()

    monthly_avg_interest_df['year_month'] = (
        monthly_avg_interest_df['year'].astype(str) + '-' + 
        monthly_avg_interest_df['month'].astype(str).str.zfill(2)
    )

    monthly_avg_interest_df_clean = monthly_avg_interest_df.drop(columns=['year','month'])

    monthly_avg_interest_df_clean

  [1/23] 2023-01-01 ถึง 2023-01-31... ✓ 42 รายการ
  [2/23] 2023-02-01 ถึง 2023-02-28... ✓ 40 รายการ
  [3/23] 2023-03-01 ถึง 2023-03-31... 

KeyboardInterrupt: 

In [21]:
import requests
from datetime import datetime, timedelta
import json
import time

def get_date_ranges(start_date, end_date):
    """
    แบ่งช่วงวันที่เป็นรายเดือน (สูงสุด 31 วัน/ครั้ง)

    Parameters:
    - start_date: วันที่เริ่มต้น (format: 'YYYY-MM-DD')
    - end_date: วันที่สิ้นสุด (format: 'YYYY-MM-DD')

    Returns:
    - list ของ tuple (start, end) แต่ละช่วง
    """
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')

    ranges = []
    current_start = start

    while current_start <= end:
        # คำนวณวันสุดท้ายของเดือน
        if current_start.month == 12:
            next_month = current_start.replace(year=current_start.year + 1, month=1, day=1)
        else:
            next_month = current_start.replace(month=current_start.month + 1, day=1)

        month_end = next_month - timedelta(days=1)
        current_end = min(month_end, end)

        ranges.append((
            current_start.strftime('%Y-%m-%d'),
            current_end.strftime('%Y-%m-%d')
        ))

        current_start = next_month

    return ranges

def fetch_data(api_url, api_key, start_date, end_date):
    """
    ดึงข้อมูลจาก API ตามช่วงวันที่

    Parameters:
    - api_url: URL ของ API
    - api_key: API Key
    - start_date: วันที่เริ่มต้น
    - end_date: วันที่สิ้นสุด

    Returns:
    - response data จาก API หรือ None ถ้าเกิด error
    """
    params = {
        'start_period': start_date,
        'end_period': end_date
    }

    headers = {
        'Authorization': api_key
    }

    try:
        response = requests.get(api_url, params=params, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {start_date} to {end_date}: {e}")
        return None

def extract_details(response_data):
    """
    สกัดเฉพาะ data_detail จากข้อมูล API

    Returns:
    - list ของ data_detail
    """
    if response_data and 'result' in response_data:
        data = response_data.get('result', {}).get('data', {})
        return data.get('data_detail', [])
    return []

def fetch_monthly_data(api_url, api_key, start_date, end_date, delay=1):
    """
    ดึงข้อมูลรายเดือนและรวม data_detail ทั้งหมด

    Parameters:
    - api_url: URL ของ API
    - api_key: API Key
    - start_date: วันที่เริ่มต้น (format: 'YYYY-MM-DD')
    - end_date: วันที่สิ้นสุด (format: 'YYYY-MM-DD')
    - delay: เวลารอระหว่างการเรียก API (วินาที)

    Returns:
    - list ของ data_detail ทั้งหมด
    """
    date_ranges = get_date_ranges(start_date, end_date)
    all_details = []

    print(f"จะดึงข้อมูล {len(date_ranges)} ช่วง")

    for i, (start, end) in enumerate(date_ranges, 1):
        print(f"กำลังดึงข้อมูลช่วงที่ {i}/{len(date_ranges)}: {start} ถึง {end}")

        response_data = fetch_data(api_url, api_key, start, end)
        details = extract_details(response_data)

        if details:
            all_details.extend(details)
            print(f"  ได้ข้อมูล {len(details)} รายการ")
        else:
            print(f"  ไม่พบข้อมูล")

        # หน่วงเวลาระหว่างการเรียก API
        if i < len(date_ranges):
            time.sleep(delay)

    print(f"\nรวมทั้งหมด: {len(all_details)} รายการ")
    return all_details


# ตัวอย่างการใช้งาน
if __name__ == "__main__":
    # ใส่ URL ของ API และ API Key
    API_URL = "https://gateway.api.bot.or.th/LoanRate/v2/avg_loan_rate/"
    API_KEY = "eyJvcmciOiI2NzM1NzgwZWM4YzFlYjAwMDEyYTM3NzEiLCJpZCI6ImRiYzQyMzA5ZmRiZTQ3YzZiZWZjZDdkYTFmMGU4ZjNiIiwiaCI6Im11cm11cjEyOCJ9"

    # กำหนดช่วงวันที่
    start_date = "2023-01-01"
    end_date = "2024-11-30"

    # ดึงข้อมูล
    data_details = fetch_monthly_data(API_URL, API_KEY, start_date, end_date, delay=1)

    # บันทึกข้อมูลเป็น JSON
    with open('loan_rates_data.json', 'w', encoding='utf-8') as f:
        json.dump(data_details, f, ensure_ascii=False, indent=2)

    print(f"\nบันทึกข้อมูลลงไฟล์ loan_rates_data.json เรียบร้อย")

    # แสดงตัวอย่างข้อมูล 2 รายการแรก
    if data_details:
        print("\nตัวอย่างข้อมูล:")
        print(json.dumps(data_details[:2], ensure_ascii=False, indent=2))

จะดึงข้อมูล 23 ช่วง
กำลังดึงข้อมูลช่วงที่ 1/23: 2023-01-01 ถึง 2023-01-31
  ได้ข้อมูล 42 รายการ
กำลังดึงข้อมูลช่วงที่ 2/23: 2023-02-01 ถึง 2023-02-28
  ได้ข้อมูล 40 รายการ
กำลังดึงข้อมูลช่วงที่ 3/23: 2023-03-01 ถึง 2023-03-31
  ได้ข้อมูล 44 รายการ
กำลังดึงข้อมูลช่วงที่ 4/23: 2023-04-01 ถึง 2023-04-30
  ได้ข้อมูล 34 รายการ


KeyboardInterrupt: 

In [28]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import json
import time
import os

def get_date_ranges(start_date, end_date):
        """แบ่งช่วงวันที่เป็นรายเดือน"""
        start = datetime.strptime(start_date, '%Y-%m-%d')
        end = datetime.strptime(end_date, '%Y-%m-%d')
        ranges = []
        current_start = start

        while current_start <= end:
            # คำนวณวันสุดท้ายของเดือน
            if current_start.month == 12:
                next_month = current_start.replace(year=current_start.year + 1, month=1, day=1)
            else:
                next_month = current_start.replace(month=current_start.month + 1, day=1)

            month_end = next_month - timedelta(days=1)
            current_end = min(month_end, end)

            ranges.append((
                current_start.strftime('%Y-%m-%d'),
                current_end.strftime('%Y-%m-%d')
            ))

            current_start = next_month

        return ranges

    def fetch_data(api_url, api_key, start_date, end_date):
        """เรียก API เพื่อดึงข้อมูล"""
        params = {
            'start_period': start_date,
            'end_period': end_date
        }
        headers = {
            'Authorization': api_key
        }

        try:
            response = requests.get(api_url, params=params, headers=headers)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {start_date} to {end_date}: {e}")
            return None

    def extract_details(response_data):
        """แยกข้อมูลจาก response"""
        if response_data and 'result' in response_data:
            data = response_data.get('result', {}).get('data', {})
            return data.get('data_detail', [])
        return []

    def fetch_monthly_data(api_url, api_key, start_date, end_date, delay=1):
        """ดึงข้อมูลทีละเดือน"""
        date_ranges = get_date_ranges(start_date, end_date)
        all_details = []

        print(f"จะดึงข้อมูล {len(date_ranges)} ช่วง")

        for i, (start, end) in enumerate(date_ranges, 1):
            response_data = fetch_data(api_url, api_key, start, end)
            details = extract_details(response_data)

            if details:
                all_details.extend(details)
                print(f"ช่วงที่ {i}: {start} ถึง {end} - พบ {len(details)} รายการ")
            else:
                print(f"ช่วงที่ {i}: {start} ถึง {end} - ไม่พบข้อมูล")

            # หน่วงเวลาระหว่างการเรียก API
            if i < len(date_ranges):
                time.sleep(delay)

        print(f"\nรวมทั้งหมด: {len(all_details)} รายการ")
        return all_details

    # กำหนดพารามิเตอร์
    API_URL = BOT_INT_AVG
    API_KEY = INT_API_KEY
    start_date = "2023-01-01"
    end_date = "2024-11-30"

    # ดึงข้อมูล
    data_details = fetch_monthly_data(API_URL, API_KEY, start_date, end_date, delay=1)

    # Clean ข้อมูล int
    interest_df = pd.DataFrame(data_details)
    numeric_cols = ['mor', 'mlr', 'mrr', 'ceiling_rate', 'default_rate', 'creditcard_min', 'creditcard_max']
    
    for col in numeric_cols:
        if col in interest_df.columns:
            interest_df[col] = pd.to_numeric(interest_df[col], errors='coerce')

    # เพิ่ม year และ month จาก period
    interest_df['period'] = pd.to_datetime(interest_df['period'])
    interest_df['year'] = interest_df['period'].dt.year
    interest_df['month'] = interest_df['period'].dt.month

    # Group and aggregate
    monthly_avg_interest_df = interest_df.groupby(['year', 'month'])[numeric_cols].mean().reset_index()
    monthly_avg_interest_df['year_month'] = (
        monthly_avg_interest_df['year'].astype(str) + '-' + 
        monthly_avg_interest_df['month'].astype(str).str.zfill(2)
    )
    monthly_avg_interest_df_clean = monthly_avg_interest_df.drop(columns=['year', 'month'])

    # เซฟไฟล์ parquet
    monthly_avg_interest_df_clean



IndentationError: unindent does not match any outer indentation level (<string>, line 34)

In [ ]:
interest_df = pd.DataFrame(data_details)
interest_df


,period,name_th,name_eng,mor,mlr,mrr,ceiling_rate,default_rate,creditcard_min,creditcard_max
0,2023-01-03,เฉลี่ยของธนาคารพาณิชย์จดทะเบียนในประเทศ,Average of Commercial Banks registered in Thai...,7.1449,6.7540,7.1947,20.4641,22.6159,16.0000,16.0000
1,2023-01-03,เฉลี่ยของสาขาธนาคารต่างประเทศ,Average of Foreign Bank Branches,7.4250,6.9159,8.4250,16.7855,22.2391,16.0000,16.0000
2,2023-01-04,เฉลี่ยของธนาคารพาณิชย์จดทะเบียนในประเทศ,Average of Commercial Banks registered in Thai...,7.1685,6.7775,7.2213,20.4641,22.6394,16.0000,16.0000
3,2023-01-04,เฉลี่ยของสาขาธนาคารต่างประเทศ,Average of Foreign Bank Branches,7.4250,6.9159,8.4250,16.7855,22.2391,16.0000,16.0000
4,2023-01-05,เฉลี่ยของธนาคารพาณิชย์จดทะเบียนในประเทศ,Average of Commercial Banks registered in Thai...,7.1832,6.7922,7.2380,20.4641,22.6394,16.0000,16.0000
...,...,...,...,...,...,...,...,...,...,...
931,2024-11-27,เฉลี่ยของสาขาธนาคารต่างประเทศ,Average of Foreign Bank Branches,7.0545,6.6659,7.1208,14.7864,18.1273,0.0000,0.0000
932,2024-11-28,เฉลี่ยของธนาคารพาณิชย์จดทะเบียนในประเทศ,Average of Commercial Banks registered in Thai...,7.7703,7.5579,7.9140,20.7835,22.9532,16.0000,16.0000
933,2024-11-28,เฉลี่ยของสาขาธนาคารต่างประเทศ,Average of Foreign Bank Branches,7.0545,6.6659,7.1208,14.7864,18.1273,0.0000,0.0000
934,2024-11-29,เฉลี่ยของธนาคารพาณิชย์จดทะเบียนในประเทศ,Average of Commercial Banks registered in Thai...,7.7703,7.5579,7.9140,20.7835,22.9532,16.0000,16.0000


# Task
Analyze the interest rate data in the DataFrame `CPI` by performing the following steps:
1. Remove rows where the value in the 'name_th' column is 'เฉลี่ยของสาขาธนาคารต่างประเทศ'.
2. Calculate the average of the numeric columns (MOR, MLR, MRR, etc.) for each month.
3. Create a new column named 'year_month' in 'yyyy-mm' format representing the year and month of each row.
4. Display the resulting DataFrame.

## Rename dataframe

### Subtask:
Rename the DataFrame containing interest data from `CPI` to `interest_df` to avoid confusion with the CPI data.


**Reasoning**:
The subtask is to rename the DataFrame `CPI` to `interest_df`. This can be done by simply assigning the `CPI` DataFrame to a new variable named `interest_df`.



In [ ]:
interest_df = CPI

## Filter data

### Subtask:
Remove rows where 'name_th' is 'เฉลี่ยของสาขาธนาคารต่างประเทศ'.


**Reasoning**:
Filter the `interest_df` to remove rows where 'name_th' is 'เฉลี่ยของสาขาธนาคารต่างประเทศ'.



In [ ]:
interest_df = interest_df[interest_df['name_th'] == 'เฉลี่ยของธนาคารพาณิชย์จดทะเบียนในประเทศ']

**Reasoning**:
The error indicates that `interest_df` is a list, not a DataFrame. I need to convert the list of dictionaries to a DataFrame first before filtering.



In [ ]:
display(interest_df.head())

,period,name_th,name_eng,mor,mlr,mrr,ceiling_rate,default_rate,creditcard_min,creditcard_max,year,month
0,2023-01-03,เฉลี่ยของธนาคารพาณิชย์จดทะเบียนในประเทศ,Average of Commercial Banks registered in Thai...,7.1449,6.7540,7.1947,20.4641,22.6159,16.0000,16.0000,2023,1
2,2023-01-04,เฉลี่ยของธนาคารพาณิชย์จดทะเบียนในประเทศ,Average of Commercial Banks registered in Thai...,7.1685,6.7775,7.2213,20.4641,22.6394,16.0000,16.0000,2023,1
4,2023-01-05,เฉลี่ยของธนาคารพาณิชย์จดทะเบียนในประเทศ,Average of Commercial Banks registered in Thai...,7.1832,6.7922,7.2380,20.4641,22.6394,16.0000,16.0000,2023,1
6,2023-01-06,เฉลี่ยของธนาคารพาณิชย์จดทะเบียนในประเทศ,Average of Commercial Banks registered in Thai...,7.1832,6.7922,7.2380,20.4641,22.6394,16.0000,16.0000,2023,1
8,2023-01-09,เฉลี่ยของธนาคารพาณิชย์จดทะเบียนในประเทศ,Average of Commercial Banks registered in Thai...,7.1832,6.7922,7.2380,20.4641,22.6394,16.0000,16.0000,2023,1


**Reasoning**:
The traceback shows a KeyError for 'name_th', indicating that the column name might be incorrect after creating the DataFrame. I will display the columns of the `interest_df` to verify the column names.



In [ ]:
display(interest_df.columns)

Index(['period', 'name_th', 'name_eng', 'mor', 'mlr', 'mrr', 'ceiling_rate',
       'default_rate', 'creditcard_min', 'creditcard_max'],
      dtype='object')

**Reasoning**:
The previous attempt to create a DataFrame from `interest_df` (which was a list) used incorrect column names. The `CPI` DataFrame, which `interest_df` was assigned from, has the correct data and column names based on the previous successful execution. I will re-assign `interest_df` from the correct `CPI` DataFrame and then perform the filtering step.



## Convert data types

### Subtask:
Convert relevant columns (MOR, MLR, MRR, etc.) to numeric types.


**Reasoning**:
Convert the relevant columns in the interest_df DataFrame to numeric types to prepare for calculations.



In [ ]:
numeric_cols = ['mor', 'mlr', 'mrr', 'ceiling_rate', 'default_rate', 'creditcard_min', 'creditcard_max']
for col in numeric_cols:
    interest_df[col] = pd.to_numeric(interest_df[col], errors='coerce')

display(interest_df.dtypes)

/tmp/ipython-input-1542987239.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interest_df[col] = pd.to_numeric(interest_df[col], errors='coerce')


,0
period,datetime64[ns]
name_th,object
name_eng,object
mor,float64
mlr,float64
mrr,float64
ceiling_rate,float64
default_rate,float64
creditcard_min,float64
creditcard_max,float64


**Reasoning**:
The previous command failed because the column 'mor' was not found in the `interest_df` DataFrame. I need to inspect the columns of `interest_df` to identify the correct numeric columns and then attempt the conversion again.



## Extract year and month

### Subtask:
Create new 'year' and 'month' columns from the 'period' column.


**Reasoning**:
Convert the 'period' column to datetime objects and extract the year and month to create new 'year' and 'month' columns.



In [ ]:
interest_df['period'] = pd.to_datetime(interest_df['period'])
interest_df['year'] = interest_df['period'].dt.year
interest_df['month'] = interest_df['period'].dt.month
display(interest_df.head())

/tmp/ipython-input-4293796748.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interest_df['period'] = pd.to_datetime(interest_df['period'])
/tmp/ipython-input-4293796748.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interest_df['year'] = interest_df['period'].dt.year
/tmp/ipython-input-4293796748.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

,period,name_th,name_eng,mor,mlr,mrr,ceiling_rate,default_rate,creditcard_min,creditcard_max,year,month
0,2023-01-03,เฉลี่ยของธนาคารพาณิชย์จดทะเบียนในประเทศ,Average of Commercial Banks registered in Thai...,7.1449,6.7540,7.1947,20.4641,22.6159,16.0000,16.0000,2023,1
2,2023-01-04,เฉลี่ยของธนาคารพาณิชย์จดทะเบียนในประเทศ,Average of Commercial Banks registered in Thai...,7.1685,6.7775,7.2213,20.4641,22.6394,16.0000,16.0000,2023,1
4,2023-01-05,เฉลี่ยของธนาคารพาณิชย์จดทะเบียนในประเทศ,Average of Commercial Banks registered in Thai...,7.1832,6.7922,7.2380,20.4641,22.6394,16.0000,16.0000,2023,1
6,2023-01-06,เฉลี่ยของธนาคารพาณิชย์จดทะเบียนในประเทศ,Average of Commercial Banks registered in Thai...,7.1832,6.7922,7.2380,20.4641,22.6394,16.0000,16.0000,2023,1
8,2023-01-09,เฉลี่ยของธนาคารพาณิชย์จดทะเบียนในประเทศ,Average of Commercial Banks registered in Thai...,7.1832,6.7922,7.2380,20.4641,22.6394,16.0000,16.0000,2023,1


## Group and aggregate

### Subtask:
Group the data by year and month, and calculate the average for each numeric column.


**Reasoning**:
Define the list of numeric columns, group the DataFrame by year and month, calculate the mean of the numeric columns for each group, reset the index, and store the result in a new DataFrame.



In [ ]:
numeric_cols = ['mor', 'mlr', 'mrr', 'ceiling_rate', 'default_rate', 'creditcard_min', 'creditcard_max']
monthly_avg_interest_df = interest_df.groupby(['year', 'month'])[numeric_cols].mean().reset_index()

**Reasoning**:
Display the resulting DataFrame to verify the aggregation.



In [ ]:
display(monthly_avg_interest_df)

,year,month,mor,mlr,mrr,ceiling_rate,default_rate,creditcard_min,creditcard_max
0,2023,1,7.214957,6.825014,7.273390,20.477529,22.653533,16.0,16.0
1,2023,2,7.360025,7.035260,7.422810,20.650890,22.854100,16.0,16.0
2,2023,3,7.374500,7.050968,7.431605,20.655300,22.854100,16.0,16.0
3,2023,4,7.456106,7.138547,7.506241,20.668806,22.868324,16.0,16.0
4,2023,5,7.556470,7.238570,7.614690,20.690575,22.880575,16.0,16.0
5,2023,6,7.676524,7.364033,7.756490,20.711481,22.910057,16.0,16.0
6,2023,7,7.758200,7.453200,7.803300,20.737100,22.937600,16.0,16.0
7,2023,8,7.788333,7.483614,7.837471,20.799733,23.009619,16.0,16.0
8,2023,9,7.770700,7.465700,7.817490,20.767100,22.968305,16.0,16.0
9,2023,10,7.930760,7.629825,8.007335,20.794570,22.982070,16.0,16.0


## Create 'year month' column

### Subtask:
Create a new column named 'year_month' in 'yyyy-mm' format from the year and month.


**Reasoning**:
Create the 'year_month' column by concatenating the 'year' and zero-padded 'month' columns and display the result.



In [ ]:
monthly_avg_interest_df['year_month'] = monthly_avg_interest_df['year'].astype(str) + '-' + monthly_avg_interest_df['month'].astype(str).str.zfill(2)
display(monthly_avg_interest_df.head())

,year,month,mor,mlr,mrr,ceiling_rate,default_rate,creditcard_min,creditcard_max,year_month
0,2023,1,7.214957,6.825014,7.273390,20.477529,22.653533,16.0,16.0,2023-01
1,2023,2,7.360025,7.035260,7.422810,20.650890,22.854100,16.0,16.0,2023-02
2,2023,3,7.374500,7.050968,7.431605,20.655300,22.854100,16.0,16.0,2023-03
3,2023,4,7.456106,7.138547,7.506241,20.668806,22.868324,16.0,16.0,2023-04
4,2023,5,7.556470,7.238570,7.614690,20.690575,22.880575,16.0,16.0,2023-05


In [ ]:
monthly_avg_interest_df

,year,month,mor,mlr,mrr,ceiling_rate,default_rate,creditcard_min,creditcard_max,year_month
0,2023,1,7.214957,6.825014,7.273390,20.477529,22.653533,16.0,16.0,2023-01
1,2023,2,7.360025,7.035260,7.422810,20.650890,22.854100,16.0,16.0,2023-02
2,2023,3,7.374500,7.050968,7.431605,20.655300,22.854100,16.0,16.0,2023-03
3,2023,4,7.456106,7.138547,7.506241,20.668806,22.868324,16.0,16.0,2023-04
4,2023,5,7.556470,7.238570,7.614690,20.690575,22.880575,16.0,16.0,2023-05
5,2023,6,7.676524,7.364033,7.756490,20.711481,22.910057,16.0,16.0,2023-06
6,2023,7,7.758200,7.453200,7.803300,20.737100,22.937600,16.0,16.0,2023-07
7,2023,8,7.788333,7.483614,7.837471,20.799733,23.009619,16.0,16.0,2023-08
8,2023,9,7.770700,7.465700,7.817490,20.767100,22.968305,16.0,16.0,2023-09
9,2023,10,7.930760,7.629825,8.007335,20.794570,22.982070,16.0,16.0,2023-10


In [ ]:
monthly_avg_interest_df_clean = monthly_avg_interest_df.drop(columns=['year','month'])
monthly_avg_interest_df_clean

,mor,mlr,mrr,ceiling_rate,default_rate,creditcard_min,creditcard_max,year_month
0,7.214957,6.825014,7.273390,20.477529,22.653533,16.0,16.0,2023-01
1,7.360025,7.035260,7.422810,20.650890,22.854100,16.0,16.0,2023-02
2,7.374500,7.050968,7.431605,20.655300,22.854100,16.0,16.0,2023-03
3,7.456106,7.138547,7.506241,20.668806,22.868324,16.0,16.0,2023-04
4,7.556470,7.238570,7.614690,20.690575,22.880575,16.0,16.0,2023-05
5,7.676524,7.364033,7.756490,20.711481,22.910057,16.0,16.0,2023-06
6,7.758200,7.453200,7.803300,20.737100,22.937600,16.0,16.0,2023-07
7,7.788333,7.483614,7.837471,20.799733,23.009619,16.0,16.0,2023-08
8,7.770700,7.465700,7.817490,20.767100,22.968305,16.0,16.0,2023-09
9,7.930760,7.629825,8.007335,20.794570,22.982070,16.0,16.0,2023-10


# Merge

In [ ]:
CPI_INT_df = CPI_clean.merge(monthly_avg_interest_df_clean, how="left", left_on="year_month", right_on="year_month")
CPI_INT_df

,base_year,price_index,mon,yoy,aoa,year_month,mor,mlr,mrr,ceiling_rate,default_rate,creditcard_min,creditcard_max
0,2019,109.54,0.24,4.69,4.69,2023-01,7.214957,6.825014,7.273390,20.477529,22.653533,16.0,16.0
1,2019,109.26,-0.26,3.87,4.28,2023-02,7.360025,7.035260,7.422810,20.650890,22.854100,16.0,16.0
2,2019,108.98,-0.26,2.93,3.83,2023-03,7.374500,7.050968,7.431605,20.655300,22.854100,16.0,16.0
3,2019,109.17,0.17,2.83,3.57,2023-04,7.456106,7.138547,7.506241,20.668806,22.868324,16.0,16.0
4,2019,108.68,-0.45,0.87,3.03,2023-05,7.556470,7.238570,7.614690,20.690575,22.880575,16.0,16.0
5,2019,109.22,0.50,0.28,2.56,2023-06,7.676524,7.364033,7.756490,20.711481,22.910057,16.0,16.0
6,2019,109.12,-0.09,0.47,2.26,2023-07,7.758200,7.453200,7.803300,20.737100,22.937600,16.0,16.0
7,2019,109.64,0.48,0.75,2.07,2023-08,7.788333,7.483614,7.837471,20.799733,23.009619,16.0,16.0
8,2019,109.15,-0.45,0.07,1.85,2023-09,7.770700,7.465700,7.817490,20.767100,22.968305,16.0,16.0
9,2019,108.98,-0.16,-0.51,1.59,2023-10,7.930760,7.629825,8.007335,20.794570,22.982070,16.0,16.0
